In [2]:
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px

# Load the data using pandas
data = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/historical_automobile_sales.csv')

# Initialize the Dash app
app = dash.Dash(__name__)

# Set the title of the dashboard
app.title = "Automobile Statistics Dashboard"

#---------------------------------------------------------------------------------
# Create the dropdown menu options
dropdown_options = [
    {'label': 'Yearly Statistics', 'value': 'Yearly Statistics'},
    {'label': 'Recession Period Statistics', 'value': 'Recession Period Statistics'}
]
# List of years 
year_list = [i for i in range(1980, 2024, 1)]
#---------------------------------------------------------------------------------------
# Create the layout of the app
app.layout = html.Div([
    html.H1("Automobile Statistics Dashboard", style={"textAlign": "center", "color": "#503D36", "fontSize": 24}),
    html.Label("Select Statistics:"),
    dcc.Dropdown(
        id='dropdown-statistics',
        options=[
            {'label': 'Yearly Statistics', 'value': 'Yearly Statistics'},
            {'label': 'Recession Period Statistics', 'value': 'Recession Period Statistics'}
        ],
        value='Yearly Statistics',  # Default selection
    ),
    dcc.Dropdown(
        id='select-year',
        options=[{'label': i, 'value': i} for i in range(1980, 2024, 1)],
        disabled=True,  # Initially disabled
    ),
    html.Div(id='output-container'),
])

#TASK 2.4: Creating Callbacks
# Define the callback function to update the input container based on the selected statistics
@app.callback(
                Output(component_id='select-year', component_property='disabled'),
                Input('dropdown-statistics', 'value'))

def update_input_container(selected_statistic):
    if selected_statistic =='Yearly Statistics': 
        return False
    else: 
        return True
#Callback for plotting
# Define the callback function to update the input container based on the selected statistics
@app.callback(
    Output(component_id='output-container', component_property='children'),
    [Input(component_id='select-year', component_property='value'), Input(component_id='dropdown-statistics', component_property='value')])


def update_output_container(selected_year, selected_statistic):
    if selected_statistic == 'Recession Period Statistics':
        # Filter the data for recession periods 
        recession_data = data[data['Recession'] == 1]
        
#TASK 2.5: Create and display graphs for Recession Report Statistics

#Plot 1 Automobile sales fluctuate over Recession Period (year wise)
        # use groupby to create relevant data for plotting
        yearly_rec=recession_data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        R_chart1 = dcc.Graph(
            figure=px.line(yearly_rec, 
                x='Year',
                y='Automobile_Sales',
                title="Average Automobile Sales fluctuation over Recession Period"))

#Plot 2 Calculate the average number of vehicles sold by vehicle type       
        # use groupby to create relevant data for plotting
        average_sales = recession_data.groupby(['Year','Vehicle_Type'], as_index=False)['Automobile_Sales'].sum()
        average_sales.set_index('Year', inplace=True)
        df_Mline = average_sales.groupby(['Vehicle_Type'])['Automobile_Sales']
        df_Mline = df_Mline.mean().reset_index()                         
        R_chart2  = dcc.Graph(figure=px.line(df_Mline,x='Vehicle_Type',y='Automobile_Sales',title='Sales Trend Vehicle-wise during Recession'))
        
# Plot 3 Pie chart for total expenditure share by vehicle type during recessions
        # use groupby to create relevant data for plotting
        exp_rec= recession_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum()
        R_chart3 = dcc.Graph(
                    figure=px.pie(exp_rec,
                    values=exp_rec.values,
                    names=exp_rec.index,
                    title="Advertising Expenditure by Vehicle Type"
                )
        )

# Plot 4 bar chart for the effect of unemployment rate on vehicle type and sales
        R_chart4 = dcc.Graph(
        figure=px.bar(
        recession_data,
        x='unemployment_rate',
        y='Automobile_Sales',
        color='Vehicle_Type',
        title="Effect of Unemployment Rate on Vehicle Type and Sales",
        labels={'Vehicle_Type': 'Vehicle Type', 'Sales': 'Sales'},
        height=400  # Set the height of the chart (adjust as needed)
    )
)
        return [
            html.Div(className='chart-item', children=[html.Div(R_chart1),html.Div(R_chart2)],style={'margin-top': '20px', 'text-align': 'center'}),
            html.Div(className='chart-item', children=[html.Div(R_chart3),html.Div(R_chart4)],style={'margin-top': '20px', 'text-align': 'center'})
            ]

# TASK 2.6: Create and display graphs for Yearly Report Statistics
 # Yearly Statistic Report Plots                             
    elif (selected_statistic=='Yearly Statistics') and selected_year:
        yearly_data = data[data['Year'] ==selected_year]
                              
#TASK 2.5: Creating Graphs Yearly data
                              
#plot 1 Yearly Automobile sales using line chart for the whole period.
        yas= data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        Y_chart1 = dcc.Graph(figure=px.line(yas, 
                x='Year',
                y='Automobile_Sales',
                title="Average Automobile Sales fluctuation over the years"))
            
# Plot 2 Total Monthly Automobile sales using line chart.
        Y_chart2 = dcc.Graph(figure=px.line(yearly_data, 
                x='Month',
                y='Automobile_Sales',
                title="Total Automobile Sales fluctuation over the months"))

            # Plot bar chart for average number of vehicles sold during the given year
        avr_vdata=yearly_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
        Y_chart3 = dcc.Graph(figure=px.bar(
        avr_vdata,
        x='Vehicle_Type',
        y='Automobile_Sales',
        title=f'Average Vehicles Sold by Vehicle Type in the year {selected_year}',
        labels={'Vehicle_Type': 'Vehicle Type', 'Automobile_Sales': 'Average Sales'},
    ))

            # Total Advertisement Expenditure for each vehicle using pie chart
        exp_data=yearly_data.groupby('Vehicle_Type')['Automobile_Sales'].sum().reset_index()
        Y_chart4 = dcc.Graph(
                    figure=px.pie(exp_data,
                    values='Automobile_Sales',
                    names='Vehicle_Type',
                    title="Advertising Expenditure by Vehicle Type"
                )
        )

#TASK 2.6: Returning the graphs for displaying Yearly data
        return [
                html.Div(className='y_chart-item', children=[html.Div(Y_chart1),html.Div(Y_chart2)],style={'display': 'flex'}),
                html.Div(className='y_chart-item', children=[html.Div(Y_chart3),html.Div(Y_chart4)],style={'display': 'flex'})
                ]
        
    else:
        return None

# Run the Dash app
if __name__ == '__main__':
    host = '127.0.0.1'  # Default local host address
    port = 8050  # Specify the port you want to use
    print(f"Running on http://{host}:{port}")
    app.run_server(host=host, port=port, debug=True)

Running on http://127.0.0.1:8050
